# Data pre-processing

In [1]:
import dlib
import cv2
import tensorflow as tf
import glob
import numpy as np
import matplotlib.image as mpimg
face_detector = dlib.get_frontal_face_detector()

shape_predictor = dlib.shape_predictor('Files/shape_predictor_68_face_landmarks.dat')

face_recognition_model = dlib.face_recognition_model_v1('Files/dlib_face_recognition_resnet_model_v1.dat')


images = []

def create_training_inputs(data_path, container, face_detector, shape_predictor, face_recognition_model):
    for i, file_name in enumerate(glob.glob(data_path)):
        im = mpimg.imread(file_name)
        faces = face_detector(im, 1)
        shapes = [shape_predictor(im, face) for face in faces]
        if len(shapes) != 0:
            face_ids = [face_recognition_model.compute_face_descriptor(im, shape, 1) for shape in shapes]
            face_ids_array = np.array(face_ids)
            face_ids_list = face_ids_array.tolist()[0]
            container.append(face_ids_list)
        else:
            print ("Cannot detect face in: ", i)
        
create_training_inputs("Training_Data/Khoi/*.jpg", images, face_detector, shape_predictor, face_recognition_model)
create_training_inputs("Training_Data/KD/*.jpg", images, face_detector, shape_predictor, face_recognition_model)

Inputs = np.array(images)

print ("Inputs shape: ",Inputs.shape)

C:\Users\kedam\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


ValueError: Only know how to handle extensions: ['png']; with Pillow installed matplotlib can handle more images

In [2]:
import csv
with open('Files/Inputs.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(Inputs)

NameError: name 'Inputs' is not defined

In [ ]:
import csv
import numpy as np

images = []

with open('Files/Inputs.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        images.append(row)
        
Inputs = np.array(images).astype(float)
print ("Inputs shape recovered: ", Inputs.shape)


In [ ]:
labels = []

khoi = [1, 0]
kd = [0, 1]

def create_training_labels(data_path, container, label):
    for file_name in glob.glob(data_path):
        container.append(label)
        
create_training_labels("Training_Data/Khoi/*.jpg", labels, khoi)
create_training_labels("Training_Data/KD/*.jpg", labels, kd)

Labels = np.array(labels)

print ("Labels shape: ", Labels.shape)


# Build Computational Graph - Tensorflow

In [3]:
import tensorflow as tf

# Hyperparameter
hidden_size1 = 200
hidden_size2 = 100


# Create placeholders
X  = tf.placeholder(tf.float32, [None, 128])
Y_ = tf.placeholder(tf.float32, [None, 2])
lr = tf.placeholder(tf.float32)
pkeep = tf.placeholder(tf.float32)

# Create variables
w1 = tf.Variable(tf.truncated_normal([128, hidden_size1], stddev=0.1), tf.float32)
b1 = tf.Variable(tf.ones([hidden_size1])/10)

w2 = tf.Variable(tf.truncated_normal([hidden_size1, hidden_size2], stddev=0.1), tf.float32)
b2 = tf.Variable(tf.ones([hidden_size2])/10)

w3 = tf.Variable(tf.truncated_normal([hidden_size2, 2], stddev=0.1), tf.float32)
b3 = tf.Variable(tf.ones([2])/10)

Y1 = tf.matmul(X, w1) + b1
Y1 = tf.nn.relu(Y1)

Y2 = tf.matmul(Y1, w2) + b2
Y2 = tf.nn.relu(Y2)

Ydrop = tf.nn.dropout(Y2, pkeep)

Ylogits = tf.matmul(Ydrop, w3) + b3
Y = tf.nn.softmax(Ylogits)

# Loss (Cross-entropy)
loss = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=Y_)

# Optimizer
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

# Accuracy
is_correct = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

# Initializer
init = tf.global_variables_initializer()
saver = tf.train.Saver()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



# Training Process

In [4]:
with tf.Session() as sess:
    sess.run(init)
    
    train_dict = {X: Inputs, Y_: Labels, lr: 0.0001, pkeep: 0.9}
    # Training
    for i in range(1000):
        sess.run(optimizer, feed_dict=train_dict)
        
        # Accuracy
        if i % 100 == 0:
            A, C = sess.run([accuracy, loss], feed_dict=train_dict)
            print ("At iteration: %s, Loss: %s, Accuracy: %s" % (i,sum(C),A))
    
    save_path = saver.save(sess, "Checkpoint/model.ckpt")
    print("Model saved in file: %s" % save_path)

NameError: name 'Inputs' is not defined

In [5]:
import numpy as np
import dlib
import matplotlib.image as mpimg


face_detector = dlib.get_frontal_face_detector()

shape_predictor = dlib.shape_predictor('Files/shape_predictor_68_face_landmarks.dat')

face_recognition_model = dlib.face_recognition_model_v1('Files/dlib_face_recognition_resnet_model_v1.dat')


known_faces = []

im = mpimg.imread("Khoi1.jpg")
faces = face_detector(im, 1)
shapes = [shape_predictor(im, face) for face in faces]
if len(shapes) != 0:
    face_ids = [face_recognition_model.compute_face_descriptor(im, shape, 1) for shape in shapes]
    face_ids_array = np.array(face_ids)
    face_ids_list = face_ids_array.tolist()[0]
    known_faces.append(face_ids_list)

im = mpimg.imread("KD1.jpg")
faces = face_detector(im, 1)
shapes = [shape_predictor(im, face) for face in faces]
if len(shapes) != 0:
    face_ids = [face_recognition_model.compute_face_descriptor(im, shape, 1) for shape in shapes]
    face_ids_array = np.array(face_ids)
    face_ids_list = face_ids_array.tolist()[0]
    known_faces.append(face_ids_list)
    

    
print (len(known_faces))


ValueError: Only know how to handle extensions: ['png']; with Pillow installed matplotlib can handle more images

# Running Application 

In [6]:
import tensorflow as tf
import numpy as np
import dlib
import cv2



face_detector = dlib.get_frontal_face_detector()

shape_predictor = dlib.shape_predictor('Files/shape_predictor_68_face_landmarks.dat')

face_recognition_model = dlib.face_recognition_model_v1('Files/dlib_face_recognition_resnet_model_v1.dat')



def detect_face(im, sess, known_faces):
    images_test = []
    faces = face_detector(im, 1)
    shapes = [shape_predictor(im, face) for face in faces]
    face_ids = [face_recognition_model.compute_face_descriptor(im, shape, 1) for shape in shapes]
    face_ids_array = np.array(face_ids)
    
#     in_test_set = True
#     unknown_faces = []
    
#     for i, face in enumerate(face_ids_array):
#         for known_face in known_faces:
#             thresh_hold = np.linalg.norm(known_face - face)
#             if thresh_hold > 0.45:
#                 in_test_set = False
#             else:
#                 in_test_set = True
#                 break
        
#         if not in_test_set:
#             font = cv2.FONT_HERSHEY_SIMPLEX
#             cv2.putText(im,"Unknown Face",(faces[i].left()-50,faces[i].top()-50), font, 2,(0,255,0),2,cv2.LINE_AA)

#             cv2.rectangle(im,(faces[i].left(),faces[i].top()),(faces[i].right(),faces[i].bottom()),(255,0,0),2)
#             unknown_faces.append(i)
            
        



    face_ids_list = face_ids_array.tolist()
    
#     print (len(face_ids_list))
#     print ("ID: ", unknown_faces)
    
#     if len(unknown_faces) != 0 and len(face_ids_list) != 0:
#         for i, index in enumerate(unknown_faces):
#             face_ids_list.pop(index - i)

    
    [images_test.append(face_id) for face_id in face_ids_list]
    
    if len(images_test) != 0:
        test_input = np.array(images_test)

        test_dict = {X: test_input, pkeep: 0.9}

        Y_predict = sess.run(Y, feed_dict=test_dict)

        for i, result in enumerate(Y_predict):
            confidence = round(result[np.argmax(result)] * 100, 2)
            if np.argmax(result) == 0:
                if np.linalg.norm(test_input[i] - known_faces[0]) < 0.45:
                    predicted_face = "Khoi face " + str(confidence) + " %"
                else:
                    predicted_face = "Unknown"
                    
            elif np.argmax(result) == 1:
                if np.linalg.norm(test_input[i] - known_faces[1]) < 0.45:
                    predicted_face = "KD face " + str(confidence) + " %"
                else:
                    predicted_face = "Unknown"

            font = cv2.FONT_HERSHEY_SIMPLEX
            
            if predicted_face == "Unknown":
                cv2.putText(im,predicted_face,(faces[i].left()-50,faces[i].top()-50), font, 2,(0,0,255),2,cv2.LINE_AA)
            else:
                cv2.putText(im,predicted_face,(faces[i].left()-50,faces[i].top()-50), font, 2,(0,255,0),2,cv2.LINE_AA)
            cv2.rectangle(im,(faces[i].left(),faces[i].top()),(faces[i].right(),faces[i].bottom()),(255,0,0),2)
    
    return im

cap = cv2.VideoCapture(0)

with tf.Session() as sess:
    saver.restore(sess, "Checkpoint/model.ckpt")
    print("Model restored.")

    while True:
        ret, frame = cap.read()
        cv2.imshow('Live stream', detect_face(frame, sess, known_faces))
        if cv2.waitKey(1) == 13:
            break

cap.release()
cv2.destroyAllWindows() 

INFO:tensorflow:Restoring parameters from Checkpoint/model.ckpt
Model restored.


IndexError: list index out of range